# Assignment 5 - Neo4j

- Allan Simonsen
- Jean-Poul Leth-Møller
- Magda Aleksandra Wawrzak
- Nina Lisakowski

 # 1. Import of libraries

In [10]:
# import pandas for structuring the data
import pandas as pd

import numpy as np

from py2neo import Graph,Node,Relationship

# 2. import of dataset from CSV

In [3]:
game_dataset = pd.read_csv('../sample-data/data/Sample_Game_1/Sample_Game_1_RawEventsData.csv', delimiter=",")

In [4]:
game_dataset.head(5)

,Team,Type,Subtype,Period,Start Frame,Start Time [s],End Frame,End Time [s],From,To,Start X,Start Y,End X,End Y
0,Away,SET PIECE,KICK OFF,1,1,0.04,0,0.00,Player19,NaN,NaN,NaN,NaN,NaN
1,Away,PASS,NaN,1,1,0.04,3,0.12,Player19,Player21,0.45,0.39,0.55,0.43
2,Away,PASS,NaN,1,3,0.12,17,0.68,Player21,Player15,0.55,0.43,0.58,0.21
3,Away,PASS,NaN,1,45,1.80,61,2.44,Player15,Player19,0.55,0.19,0.45,0.31
4,Away,PASS,NaN,1,77,3.08,96,3.84,Player19,Player21,0.45,0.32,0.49,0.47


In [5]:
game_dataset.tail(5)

,Team,Type,Subtype,Period,Start Frame,Start Time [s],End Frame,End Time [s],From,To,Start X,Start Y,End X,End Y
1740,Home,PASS,NaN,2,143361,5734.44,143483,5739.32,Player12,Player13,0.60,0.33,0.19,0.95
1741,Home,PASS,NaN,2,143578,5743.12,143593,5743.72,Player13,Player4,0.09,0.88,0.14,0.69
1742,Home,BALL LOST,INTERCEPTION,2,143598,5743.92,143618,5744.72,Player4,NaN,0.13,0.69,0.07,0.61
1743,Away,RECOVERY,BLOCKED,2,143617,5744.68,143617,5744.68,Player16,NaN,0.05,0.62,NaN,NaN
1744,Away,BALL OUT,NaN,2,143622,5744.88,143630,5745.20,Player16,NaN,0.05,0.63,0.03,1.01


In [6]:
game_dataset.shape

(1745, 14)

In [7]:
game_dataset['From'].unique()

array(['Player19', 'Player21', 'Player15', 'Player22', 'Player17',
       'Player2', 'Player16', 'Player18', 'Player9', 'Player10',
       'Player20', 'Player5', 'Player6', 'Player8', 'Player25', 'Player1',
       'Player7', 'Player4', 'Player3', 'Player24', 'Player23',
       'Player11', 'Player12', 'Player27', 'Player13', 'Player26',
       'Player14', 'Player28'], dtype=object)

In [8]:
game_dataset['Type'].unique()

array(['SET PIECE', 'PASS', 'BALL LOST', 'RECOVERY', 'CHALLENGE',
       'BALL OUT', 'SHOT', 'FAULT RECEIVED', 'CARD'], dtype=object)

# 3. Connection to database

In [11]:
graph = Graph('bolt://localhost:7687', auth=('neo4j', 'root123'))

# 4. Create Player nodes

We decided to make all player as seperate nodes.  
furtermore we decided to make each event (PASS, SET KICK, Challenge, etc) as a new node with the time property and a relationship to and from related players.
The following picture show the setup with the PASS event.
![Graph Setup](GraphSetup.png)

In [8]:
for i in game_dataset['From'].unique():
    print(i)
    try:
        new_player = Node("player",name=i)
        tx=graph.begin()
        tx.create(new_player)
        graph.commit(tx)
    except:
        print('Player already Exist')
    
    


Player19
Player21
Player15
Player22
Player17
Player2
Player16
Player18
Player9
Player10
Player20
Player5
Player6
Player8
Player25
Player1
Player7
Player4
Player3
Player24
Player23
Player11
Player12
Player27
Player13
Player26
Player14
Player28


# 5. Create Relationships and Event Nodes

In [9]:
for i in game_dataset.itertuples():
    try:
        fromPlayer = graph.nodes.match("player", name=i.From).first()
        toPlayer = graph.nodes.match("player", name=i.To).first()
        new_Event_node = Node(i.Type, name=i.Type, subtype=i.Subtype, team=i.Team, period=i.Period, starttime=i._6, endtime=i._8)
        relationshipfrom = Relationship(fromPlayer, i.Type, new_Event_node)
        tx=graph.begin()
        tx.create(new_Event_node)
        tx.create(relationshipfrom)
        if(toPlayer is not None):
            relationshipTo = Relationship(new_Event_node, i.Type, toPlayer)
            tx.create(relationshipTo)
        graph.commit(tx)
        print("Event Done")
    except BaseException as E:
        print(E)

Pandas(Index=0, Team='Away', Type='SET PIECE', Subtype='KICK OFF', Period=1, _5=1, _6=0.04, _7=0, _8=0.0, From='Player19', To=nan, _11=nan, _12=nan, _13=nan, _14=nan)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=1, Team='Away', Type='PASS', Subtype=nan, Period=1, _5=1, _6=0.04, _7=3, _8=0.12, From='Player19', To='Player21', _11=0.45, _12=0.39, _13=0.55, _14=0.43)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=2, Team='Away', Type='PASS', Subtype=nan, Period=1, _5=3, _6=0.12, _7=17, _8=0.68, From='Player21', To='Player15', _11=0.55, _12=0.43, _13=0.58, _14=0.21)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=3, Team='Away', Type='PASS', Subtype=nan, Period=1, _5=45, _6=1.8, _7=61, _8=2.44, From='Player15', To='Player19', _11=0.55, _12=0.19, _13=0.45, _14=0.31)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>

Hello3
Hello4
Event Done
Pandas(Index=37, Team='Away', Type='PASS', Subtype=nan, Period=1, _5=3780, _6=151.2, _7=3805, _8=152.2, From='Player21', To='Player20', _11=0.57, _12=0.53, _13=0.58, _14=0.73)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=38, Team='Away', Type='PASS', Subtype=nan, Period=1, _5=3829, _6=153.16, _7=3854, _8=154.16, From='Player20', To='Player22', _11=0.57, _12=0.76, _13=0.52, _14=0.98)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=39, Team='Away', Type='PASS', Subtype=nan, Period=1, _5=3882, _6=155.28, _7=3907, _8=156.28, From='Player22', To='Player17', _11=0.55, _12=0.95, _13=0.69, _14=0.87)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=40, Team='Away', Type='PASS', Subtype=nan, Period=1, _5=3930, _6=157.2, _7=3994, _8=159.76, From='Player17', To='Player25', _11=0.71, _12=0.86, _13=

Hello3
Hello4
Event Done
Pandas(Index=82, Team='Away', Type='BALL LOST', Subtype='HEAD', Period=1, _5=6779, _6=271.16, _7=6844, _8=273.76, From='Player18', To=nan, _11=0.53, _12=0.04, _13=0.57, _14=0.31)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=83, Team='Home', Type='RECOVERY', Subtype=nan, Period=1, _5=6844, _6=273.76, _7=6844, _8=273.76, From='Player6', To=nan, _11=0.54, _12=0.3, _13=nan, _14=nan)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=84, Team='Away', Type='CHALLENGE', Subtype='GROUND-WON', Period=1, _5=6863, _6=274.52, _7=6863, _8=274.52, From='Player21', To=nan, _11=0.53, _12=0.29, _13=nan, _14=nan)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=85, Team='Away', Type='RECOVERY', Subtype='INTERCEPTION', Period=1, _5=6863, _6=274.52, _7=6863, _8=274.52, From='Player21', To=nan, _11=0.53, _12=0.29, _13=nan, _14=nan)
type <class 'fl

Pandas(Index=129, Team='Away', Type='PASS', Subtype=nan, Period=1, _5=9215, _6=368.6, _7=9255, _8=370.2, From='Player23', To='Player18', _11=0.06, _12=0.0, _13=0.22, _14=0.06)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=130, Team='Away', Type='PASS', Subtype=nan, Period=1, _5=9306, _6=372.24, _7=9337, _8=373.48, From='Player18', To='Player19', _11=0.18, _12=0.11, _13=0.24, _14=0.3)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=131, Team='Away', Type='BALL LOST', Subtype='INTERCEPTION', Period=1, _5=9337, _6=373.48, _7=9356, _8=374.24, From='Player19', To=nan, _11=0.24, _12=0.3, _13=0.24, _14=0.39)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=132, Team='Home', Type='RECOVERY', Subtype='INTERCEPTION', Period=1, _5=9354, _6=374.16, _7=9354, _8=374.16, From='Player6', To=nan, _11=0.22, _12=0.36, _13=nan, _14=nan)
type <

Event Done
Pandas(Index=176, Team='Away', Type='BALL LOST', Subtype=nan, Period=1, _5=13766, _6=550.64, _7=13814, _8=552.56, From='Player15', To=nan, _11=0.64, _12=0.55, _13=0.76, _14=0.52)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=177, Team='Home', Type='RECOVERY', Subtype=nan, Period=1, _5=13814, _6=552.56, _7=13814, _8=552.56, From='Player9', To=nan, _11=0.73, _12=0.56, _13=nan, _14=nan)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=178, Team='Home', Type='CHALLENGE', Subtype='TACKLE-LOST', Period=1, _5=13843, _6=553.72, _7=13843, _8=553.72, From='Player9', To=nan, _11=0.8, _12=0.52, _13=nan, _14=nan)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=179, Team='Home', Type='BALL LOST', Subtype='THEFT', Period=1, _5=13843, _6=553.72, _7=13843, _8=553.72, From='Player9', To=nan, _11=0.8, _12=0.52, _13=nan, _14=nan)
type <class 'float'>
Hello1


...

Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=1513, Team='Away', Type='RECOVERY', Subtype='INTERCEPTION', Period=2, _5=123920, _6=4956.8, _7=123920, _8=4956.8, From='Player16', To=nan, _11=0.36, _12=0.37, _13=nan, _14=nan)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=1514, Team='Away', Type='PASS', Subtype=nan, Period=2, _5=124005, _6=4960.2, _7=124045, _8=4961.8, From='Player16', To='Player25', _11=0.15, _12=0.25, _13=0.01, _14=0.32)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=1515, Team='Away', Type='BALL LOST', Subtype='INTERCEPTION', Period=2, _5=124051, _6=4962.04, _7=124146, _8=4965.84, From='Player25', To=nan, _11=0.01, _12=0.32, _13=0.45, _14=0.43)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=1516, Team='Home', Type='RECOVERY', Subtype='INTERCEPTION', Period=2, _5=124147, _6=4965.88, _7=124147

Event Done
Pandas(Index=1566, Team='Home', Type='PASS', Subtype='HEAD', Period=2, _5=127891, _6=5115.64, _7=127911, _8=5116.44, From='Player3', To='Player12', _11=0.85, _12=0.55, _13=0.81, _14=0.6)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=1567, Team='Home', Type='PASS', Subtype=nan, Period=2, _5=127919, _6=5116.76, _7=127954, _8=5118.16, From='Player12', To='Player4', _11=0.82, _12=0.61, _13=0.89, _14=0.75)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=1568, Team='Home', Type='PASS', Subtype=nan, Period=2, _5=127959, _6=5118.36, _7=128009, _8=5120.36, From='Player4', To='Player8', _11=0.89, _12=0.77, _13=0.66, _14=0.94)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=1569, Team='Home', Type='PASS', Subtype=nan, Period=2, _5=128058, _6=5122.32, _7=128088, _8=5123.52, From='Player8', To='Player13', _11=0.

Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=1610, Team='Home', Type='PASS', Subtype=nan, Period=2, _5=131256, _6=5250.24, _7=131319, _8=5252.76, From='Player3', To='Player7', _11=0.54, _12=0.78, _13=0.45, _14=0.15)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=1611, Team='Home', Type='PASS', Subtype=nan, Period=2, _5=131399, _6=5255.96, _7=131424, _8=5256.96, From='Player7', To='Player14', _11=0.39, _12=0.13, _13=0.31, _14=0.06)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=1612, Team='Home', Type='PASS', Subtype=nan, Period=2, _5=131429, _6=5257.16, _7=131449, _8=5257.96, From='Player14', To='Player7', _11=0.31, _12=0.06, _13=0.37, _14=0.14)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=1613, Team='Home', Type='PASS', Subtype=nan, Period=2, _5=131466, _6=5258.64

Hello4
Event Done
Pandas(Index=1659, Team='Away', Type='PASS', Subtype=nan, Period=2, _5=135494, _6=5419.76, _7=135524, _8=5420.96, From='Player17', To='Player28', _11=0.81, _12=0.79, _13=0.75, _14=0.66)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=1660, Team='Away', Type='BALL LOST', Subtype='INTERCEPTION', Period=2, _5=135531, _6=5421.24, _7=135561, _8=5422.44, From='Player28', To=nan, _11=0.75, _12=0.67, _13=0.87, _14=0.32)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=1661, Team='Home', Type='RECOVERY', Subtype='INTERCEPTION', Period=2, _5=135566, _6=5422.64, _7=135566, _8=5422.64, From='Player7', To=nan, _11=0.88, _12=0.35, _13=nan, _14=nan)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=1662, Team='Home', Type='PASS', Subtype=nan, Period=2, _5=135585, _6=5423.4, _7=135595, _8=5423.8, From='Player7', To='Player14', _11=0.87, _

Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=1713, Team='Away', Type='RECOVERY', Subtype='INTERCEPTION', Period=2, _5=140675, _6=5627.0, _7=140675, _8=5627.0, From='Player20', To=nan, _11=0.41, _12=0.15, _13=nan, _14=nan)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=1714, Team='Away', Type='PASS', Subtype=nan, Period=2, _5=140729, _6=5629.16, _7=140744, _8=5629.76, From='Player20', To='Player28', _11=0.42, _12=0.04, _13=0.5, _14=0.15)
type <class 'str'>
Hello1
toPlayer <class 'py2neo.data.Node'>
HEllo2
Hello3
Hello4
Event Done
Pandas(Index=1715, Team='Home', Type='CHALLENGE', Subtype='TACKLE-FAULT-LOST', Period=2, _5=140893, _6=5635.72, _7=140893, _8=5635.72, From='Player14', To=nan, _11=0.56, _12=0.41, _13=nan, _14=nan)
type <class 'float'>
Hello1
toPlayer <class 'NoneType'>
HEllo2
Hello3
Event Done
Pandas(Index=1716, Team='Away', Type='CHALLENGE', Subtype='TACKLE-FAULT-WON', Period=2, _5=140912, _6=5636.48, 

# 6. Getting Data from the graph

## 6.1 Who is the most active player (in terms of passing and receiving the ball)?

We count the amount of PASS relationship to and from each player node

In [12]:
activity_result=graph.run("MATCH(player:player)-[event:PASS]-(:PASS) WITH player, COUNT(event) AS num ORDER BY num DESC RETURN player.name,num")
df = pd.DataFrame(activity_result)
df

,0,1
0,Player21,135
1,Player12,106
2,Player7,102
3,Player4,89
4,Player19,85
5,Player2,84
6,Player6,83
7,Player17,80
8,Player8,76
9,Player3,74


#### Answer: 
Player21 is the most active player in terms of passing and recieving the ball with a total of 135 PASS events related to him/her.

## 6.2 who has had a central role in the match?

we count the total amount of relationships each players have to find the most active player.  
one could argue that some events are worth more than others, however in this case we count the equally.

In [20]:
activity_result=graph.run("MATCH(player:player)-[event]-() WITH player, COUNT(event) AS num ORDER BY num DESC RETURN player.name,num")
df = pd.DataFrame(activity_result)
df

,0,1
0,Player21,172
1,Player19,149
2,Player7,149
3,Player8,137
4,Player12,135
5,Player17,134
6,Player6,130
7,Player2,129
8,Player4,123
9,Player3,117


#### Answer
We see that player21 has the most relationships, we therefore conclude that he/she had the biggest impact on the match

## 6.3 which players have attempted to score?

We count the amount of SHOT reltionships for each player

In [13]:
activity_result=graph.run("MATCH(player:player)-[event:SHOT]-(:SHOT) WITH player, COUNT(event) AS num ORDER BY num DESC RETURN player.name,num")
df = pd.DataFrame(activity_result)
df


,0,1
0,Player9,4
1,Player10,4
2,Player8,4
3,Player7,2
4,Player5,2
5,Player21,1
6,Player24,1
7,Player19,1
8,Player12,1
9,Player27,1


In [14]:
activity_result=graph.run("MATCH(player:player)-[event:SHOT]-(eventNode:SHOT) WHERE eventNode.subtype CONTAINS 'GOAL' WITH player, COUNT(event) AS num ORDER BY num DESC RETURN player.name,num")
df = pd.DataFrame(activity_result)
df

,0,1
0,Player9,2
1,Player10,1


#### Answer
Quite a few players have tried to score goals, player9 and player10 both succeeded in goals and the match ended 3-0

## 6.4  which team has kept the ball longer?

We add up the start minus the end time and sum them together for each team

In [24]:
activity_result_away=graph.run("MATCH (p:player)-[:PASS]->(e:PASS)-[:PASS]->(p2:player) WHERE e.team = 'Away' RETURN sum(toInteger(e.endtime) - toInteger(e.starttime))")
activity_result_away



sum(toInteger(e.endtime) - toInteger(e.starttime))
491


In [26]:
activity_result_Home=graph.run("MATCH (p:player)-[:PASS]->(e:PASS)-[:PASS]->(p2:player) WHERE e.team = 'Home' RETURN sum(toInteger(e.endtime) - toInteger(e.starttime))")
activity_result_Home

sum(toInteger(e.endtime) - toInteger(e.starttime))
613


#### Answer
We can see that the home team has had to ball for more time than the away team

## 6.5  is there any close ‘societies’ between players (passing the ball to each other)?

we count the amount of PASS relationships between two players

In [16]:
activity_result=graph.run("MATCH(player:player)-[event:PASS]-(:PASS)-[r2:PASS]-(toPlayer:player)WITH player, toPlayer,COUNT(event) AS num ORDER BY num DESC RETURN player.name, toPlayer.name, num")
df = pd.DataFrame(activity_result)
df.head(10)


,0,1,2
0,Player7,Player2,25
1,Player2,Player7,25
2,Player21,Player19,24
3,Player19,Player21,24
4,Player4,Player8,23
5,Player8,Player4,23
6,Player5,Player7,19
7,Player7,Player5,19
8,Player22,Player21,18
9,Player21,Player22,18


#### Answer
We can see that player7-player2, player21-19 and player4-player8 has the highest amount of passing the ball to eachother

## 6.6  how close is the connection between two specific players?

Since players dont have relationship directly to eachother we will count the amount of PASS event nodes there is between two players

In [13]:
player1= "Player21"
player2= "Player15"
activity_result=graph.run("MATCH(player:player{name:'"+ player1 +"'})-[event:PASS]-(:PASS)-[r2:PASS]-(toPlayer:player{name:'"+ player2 +"'})WITH player, toPlayer,COUNT(event) AS num ORDER BY num DESC RETURN player.name, toPlayer.name, num")
df = pd.DataFrame(activity_result)
df.head(10)


,0,1,2
0,Player21,Player15,14


#### Answer
We can see that there are 14 connection between 2 specific players, in this case player21 and player15

_______________________________________________________

In [7]:
graph.evaluate("MATCH (n) DETACH DELETE n")